In [1]:
from OddoDownload import OddoDownload
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
conn_params = {
    'ODOO_USERNAME' : 'acceso@asalvo.cl',
    'ODOO_PASSWORD' : '1234',
    'ODOO_HOSTNAME' : 'sinergia-lab-cencorep-ambiente-test-9601418.dev.odoo.com',
    'ODOO_DATABASE' : 'sinergia-lab-cencorep-ambiente-test-9601418'
}
odoo = OddoDownload(conn_params)

In [3]:
modelo = 'x_productos'
filtros = [('x_studio_unidades_de_negocio','=','SMK')]
campos = ['x_studio_sku_unidad_de_negocio','x_name','x_studio_stage_id','x_studio_variable_de_marcado','x_studio_candidato_a_analisis_fisico']
header = ['SKU unidad negocio','SKU','Etapa','EVA','Analisis fisico']
campos_fk = ['x_studio_stage_id']

odoo.getDataFromModel(modelo,filtros,campos,header,campos_fk=campos_fk)
# odoo.downloadExcel('Maestra','csv')
# print('Se ha generado el archivo Maestra.csv')

Descargando data desde x_productos
.
.
done


In [6]:
unidad_negocio = 'JUMBO'

if unidad_negocio not in ['JUMBO','SISA']:
    raise Exception('La unidad de negocio debe ser "JUMBO" o "SISA"')

if unidad_negocio=='JUMBO':
    campos_ventas_totales = ['x_studio_total_conveniencia','x_studio_total_jumbo']
    header_ventas_totales = ['TOTAL CONVENIENCIA','TOTAL JUMBO']
elif unidad_negocio=='SISA':
    campos_ventas_totales = ['x_studio_total_sisa']
    header_ventas_totales = ['TOTAL SISA']

# =========================
# DESCARGAR TABLA DE VENTAS
# =========================
modelo = 'x_ventas'
filtros = ['&',('x_studio_unidades_de_negocio','=','SMK'),('x_studio_periodo.x_name','=',2023)]
campos = ['x_studio_sku_unidad_de_negocio','x_studio_descripcin_producto','x_studio_elementos_del_producto'] + campos_ventas_totales
header = ['Producto','Producto/Descripción','lista_elementos'] + header_ventas_totales 
campos_fk = ['x_studio_producto']

ventas = odoo.getDataFromModel(modelo,filtros,campos,ret_=True,campos_fk=campos_fk, header=header)

Descargando data desde x_ventas
.
done


In [7]:



# =============================
# CONTAR ELEMENTOS A DESCARGAR
# ============================
total_elementos = []
for i in range(len(ventas)):
    elementos = eval(ventas['lista_elementos'].iloc[i])
    total_elementos += elementos
len(set(total_elementos))

# ===========================
# DESCARGAR DETALLE DE PARTES
# ===========================
modelo = 'x_materialidad'
filtros = [("id","in",total_elementos)]
campos = ['x_name','x_studio_productos_por_envase','x_studio_peso','x_studio_peso_informado','x_studio_mat',
        'x_studio_caractertica_del_material_solo_para_plsticos','x_studio_definir_otro_material','x_studio_caracterstica_reciclable',
        'x_studio_caracteristica_retornable','x_studio_peligrosidad','x_studio_categora','x_studio_cat_material']

header = ['Elementos del producto','Productos por envase','Peso','Peso informado','Material','caracterítica del material (solo para plásticos, cartón y vidrio)',
        'Definir otro material','Característica reciclable','Característica retornable','Peligrosidad','Categoría','Categoría material']

campos_fk = ['x_studio_mat']

materialidad = odoo.getDataFromModel(modelo,filtros,campos,ret_=True,drop_id=False,campos_fk=campos_fk,header=header)
materialidad['id'] = pd.to_numeric(materialidad['id'], errors='coerce')

# ===========================
# CREAR TABLA FINAL
# ===========================

header_1 = ['Producto','Producto/Descripción']
header_2 = ['Elementos del producto','Productos por envase','Peso','Peso informado','Material','caracterítica del material (solo para plásticos, cartón y vidrio)',
        'Definir otro material','Característica reciclable','Característica retornable','Peligrosidad','Categoría','Categoría material']
header_3 = header_ventas_totales
final_header = header_1+header_2+header_3

n_campos = len(final_header)
declaracion_eye = np.zeros( (len(total_elementos),n_campos),dtype='object' )

index_declaracion = 0
for i in tqdm(range(len(ventas))):                                      # POR CADA FILA EN VENTAS (tabla x_ventas)
    lista_elementos = eval(ventas.iloc[i].lista_elementos)              # OBTENGO LOS ELEMENTOS (o partes xd)              
    parte1 = ventas[header_1].iloc[i].to_numpy().reshape(1,-1)  
    parte3 = ventas[header_3].iloc[i].to_numpy().reshape(1,-1)

    for elemento in lista_elementos:                                    # POR CADA PARTE
        detalle_elemento = materialidad[ materialidad['id']==elemento ]
        detalle_elemento = detalle_elemento[header_2].to_numpy().reshape(1,-1)
        row_declaracion = np.concatenate([parte1,detalle_elemento,parte3],axis=1)

        declaracion_eye[index_declaracion] = row_declaracion            # ANADE EL ELEMENTO A LA TABLA FINAL
        index_declaracion += 1

declaracion_eye = pd.DataFrame(data=declaracion_eye,columns=final_header)
declaracion_eye = declaracion_eye[(declaracion_eye['Categoría']=='EYE Domiciliario') | (declaracion_eye['Categoría']=='EYE No domiciliario')]
declaracion_eye = declaracion_eye.replace('False','')

Descargando data desde x_materialidad
.
.
.
done


100%|██████████| 22218/22218 [01:16<00:00, 291.60it/s]


In [ ]:
if unidad_negocio=='JUMBO':
    declaracion_eye['Peso']=declaracion_eye['Peso'].astype('float')
    declaracion_eye['TOTAL JUMBO']=declaracion_eye['TOTAL JUMBO'].astype('float')
    declaracion_eye['TOTAL CONVENIENCIA']=declaracion_eye['TOTAL CONVENIENCIA'].astype('float')

    declaracion_eye['Peso total (gr)'] = (declaracion_eye['TOTAL JUMBO']+declaracion_eye['TOTAL CONVENIENCIA'])*declaracion_eye['Peso']
    declaracion_eye['Peso total (kg)'] = 1e-3*(declaracion_eye['TOTAL JUMBO']+declaracion_eye['TOTAL CONVENIENCIA'])*declaracion_eye['Peso']
    declaracion_eye['Peso total (ton)'] = 1e-6*(declaracion_eye['TOTAL JUMBO']+declaracion_eye['TOTAL CONVENIENCIA'])*declaracion_eye['Peso']


In [20]:
test  = declaracion_eye.copy()
test['Peso'] = test['Peso'].astype('float')
caso = test[test['Producto']=='68219']
caso['Peso']

caso.to_excel('caso.xlsx')

ValueError: No engine for filetype: ''